In [5]:
# Install and load necessary packages
install.packages("keras")
install.packages("tidyverse")  # For data manipulation and visualization
install.packages("caret")      # For model evaluation
install.packages("e1071")      # For preprocessing
install.packages("rmarkdown")  # For documentation
install.packages("shapper")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [6]:
library(keras)
library(tidyverse)
library(caret)
library(e1071)
library(rmarkdown)
library(shapper)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




In [7]:
# Load your dataset
data <- read_csv('/content/sample_data/Starbucks_satisfactory_survey.csv')

Rows: 113 Columns: 33
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (33): ï»¿Id, gender, age, status, income, visitNo, method, timeSpend, lo...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [8]:
str(data)

spc_tbl_ [113 × 33] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ ï»¿Id                 : num [1:113] 1 2 3 4 5 6 7 8 9 10 ...
 $ gender                : num [1:113] 1 1 0 1 0 1 1 0 1 0 ...
 $ age                   : num [1:113] 1 1 1 1 1 1 1 1 1 1 ...
 $ status                : num [1:113] 0 0 2 0 0 0 0 2 0 2 ...
 $ income                : num [1:113] 0 0 0 0 0 0 0 2 0 0 ...
 $ visitNo               : num [1:113] 3 3 2 3 2 3 3 3 3 2 ...
 $ method                : num [1:113] 0 2 0 2 2 0 0 0 1 2 ...
 $ timeSpend             : num [1:113] 1 0 1 0 1 1 0 1 0 0 ...
 $ location              : num [1:113] 0 1 2 2 1 2 0 2 2 2 ...
 $ membershipCard        : num [1:113] 0 0 0 1 1 1 0 0 0 1 ...
 $ itemPurchaseCoffee    : num [1:113] 1 1 1 1 1 1 1 1 1 1 ...
 $ itempurchaseCold      : num [1:113] 1 1 1 1 1 1 1 1 1 1 ...
 $ itemPurchasePastries  : num [1:113] 1 1 1 1 1 1 1 1 1 1 ...
 $ itemPurchaseJuices    : num [1:113] 1 1 1 1 1 1 1 1 1 1 ...
 $ itemPurchaseSandwiches: num [1:113] 1 1 1 1 1 1 1 1 1 

In [10]:
# Handle missing values
data <- na.omit(data)

In [11]:
unique_levels <- unique(data$loyal)
print(unique_levels)

[1] 0 1


In [12]:
# Split data into training and testing sets
set.seed(123) # for reproducibility
train_index <- createDataPartition(data$loyal, p=0.8, list= FALSE)
train_data <- data[train_index, ]
test_data <- data[-train_index, ]

In [13]:
# Scale numerical features
prepoc_train <- preProcess(train_data, method = c("center", "scale"))

Warning message in preProcess.default(train_data, method = c("center", "scale")):
“These variables have zero variances: itemPurchaseCoffee, itempurchaseCold, itemPurchasePastries, itemPurchaseJuices, itemPurchaseSandwiches, itemPurchaseOthers, promoMethodApp, promoMethodSoc, promoMethodEmail, promoMethodDeal, promoMethodFriend, promoMethodDisplay, promoMethodBillboard”


In [14]:
# These variables have zero variances: itemPurchaseCoffee, itempurchaseCold, itemPurchasePastries,
# itemPurchaseJuices, itemPurchaseSandwiches, itemPurchaseOthers, promoMethodApp, promoMethodSoc,
# promoMethodEmail, promoMethodDeal, promoMethodFriend, promoMethodDisplay, promoMethodBillboard
# Variables with zero variances do not contribute to the variability in the dataset
# removing these variables from the dataset

In [15]:
numeric_vars <- c('age', 'income', 'visitNo', 'spendPurchase', 'productRate', 'priceRate')
preproc_train <- preProcess(train_data[, numeric_vars], method = c("center", "scale"))
train_data[, numeric_vars] <- predict(preproc_train, train_data[, numeric_vars])
test_data[, numeric_vars] <- predict(preproc_train, test_data[, numeric_vars])

In [16]:
# Convert data to matrix format for Keras
x_train <- as.matrix(train_data[, setdiff(names(train_data), "loyal")])
y_train <- as.numeric(train_data$loyal)
x_test <- as.matrix(test_data[, setdiff(names(test_data), "loyal")])
y_test <- as.numeric(test_data$loyal)

In [17]:
# Define the Neural Network Architecture
model <- keras_model_sequential() %>%
  layer_dense(units = 64, activation = 'relu', input_shape = ncol(x_train)) %>%
  layer_dense(units = 32, activation = 'relu') %>%
  layer_dense(units = 1, activation = 'sigmoid')

In [20]:
# Compile the model
model %>% compile(
  loss = 'binary_crossentropy',
  optimizer = optimizer_rmsprop(),
  metrics = c('accuracy')
)

In [21]:
# Train the mode
history <- model %>% fit(
  x_train, y_train,
  epochs = 10,
  batch_size = 32,
  validation_split = 0.2
)

In [22]:
# Evaluate the model
evaluation <- model %>% evaluate(x_test, y_test)
print(evaluation)

     loss  accuracy 
0.4391091 0.9090909 


In [23]:
# Example of interpreting binary classification results
accuracy <- evaluation[[2]]

# Get raw predictions from the model
raw_predictions <- model %>% predict(x_test)

# Apply thresholding to obtain binary predictions
predictions <- ifelse(raw_predictions > 0.5, 1, 0)

# Ensure predictions have the same length as y_test
predictions <- predictions[1:length(y_test)]

# Compute confusion matrix
conf_mat <- table(Actual = y_test, Predicted = predictions)

In [24]:
cat("Accuracy:", accuracy*100, "\n")

Accuracy: 90.90909 


In [25]:
print(conf_mat)

      Predicted
Actual  0
     0 20
     1  2


In [28]:
# Saving the model
save_model_hdf5(model, "trained_model.h5")